# *IceFlow & icepyx*: Altimetry Time Series Tutorial
### NASA Earthdata Webinar - April 2021</b>

This tutorial demonstrates how to harmonize several NASA altimetry data products with varying temporal coverage, formats, and coordinate reference frames using the IceFlow and icepyx Python tools. 

#### Questions addressed:

#### Objectives:
1. Explore coverage and structure of the data...
2. Use IceFlow map widget to view area of interest...
3. Search and download spatiotemporally coincident data...
4. Use icepyx to subset ICESat-2 data...
5. Learn about advanced icepyx features...
6. Extract variables across all data products...
7. Visualize time series...

<b>Authors:</b><br />
<span style="font-size:larger;">Jessica Scheick</span>, *University of New Hampshire*, Durham, New Hampshire<br />
<span style="font-size:larger;">Nicholas Kotlinski & Amy Steiker</span>, *NASA National Snow and Ice Data Center DAAC*, Boulder, Colorado, USA

---

#### Running this tutorial locally

To run this notebook locally, you must first set up your computing environment. Please see the [repository readme](https://github.com/nsidc/NSIDC-Data-Tutorials#usage-with-binder) for instructions on several ways (using Binder, Docker, or Conda) to do this.

<b>Other needs:</b><br />
* IceFlow API examples
* icepyx examples
* Code to configure an ICESat-2 file to match IceFlow data dictionary

### 1. NASA's Earthdata Credentials

To access data using the *IceFlow* library and *icepyx* package, it is necessary to log into [Earthdata Login](https://urs.earthdata.nasa.gov/). To do this, enter your NASA Earthdata credentials in the next step after executing the following code cell.

**Note**: If you don't have NASA Earthdata credentials you will need to register first at the link above. An account is free and available to everyone!

In [ ]:
# This cell will prompt you for your Earthdata username and password. Press
from iceflow.ui import IceFlowUI
client = IceFlowUI()
client.display_credentials()

In [ ]:
# This cell will verify if your credentials are valid. 
# This may take a little while. If it fails, please try again.

authorized = client.authenticate()
if authorized is None:
    print('Earthdata Login not successful')
else:
    print('Earthdata Login successful!')

**Note:** If the output shows "You are logged into NASA Earthdata!", then you are ready to proceed!

---

### 2. Accessing and harmonizing data across missions
#### 2.1. Accessing Data with the *IceFlow* Access Widget
The *IceFlow* access widget is a user interface tool to visualize flightpaths from IceBridge, draw a region of interest, set spatio-temporal parameters and place data orders to the *IceFlow* API and *icepyx* package without the need to writing code.
The output of the operations performed in the widget can be seen in the log window (right-most icon at the bottom of your browser.) 
<img src='./img/log-icons.png'> or by selecting it on the View menu "Show log console"

**Note:** The access widget is currently stateless, so if you change any parameter you will have to redraw your bounding box or polygon.

In [ ]:
# Let's start with the user interface. Using 'horizontal' will add the widget inline.
client.display_map('horizontal', extra_layers=True)

#### 2.2. Accessing data with the *IceFlow* API

### ToDo: Need to explain what the itrf and epoch values are and which ones are best to utilize in conjunction with ICESat-2 analysis. 

### **Amy note:** Will we see the bbox above in the map? Or is there another way we can visualize the area?

In [ ]:
# Small example subset over Jakobshavn glacier:
my_params ={
    'datasets': ['ATM1B', 'GLAH06', 'ILVIS2', 'ATL06'],
    'start': '2008-01-01',
    'end': '2018-12-31',
    'bbox': '-49.787268,69.103805,-49.526401,69.218717',
    'itrf': 'ITRF2014',
    'epoch': '2014.1'
}

# returns a json dictionary, the request parameters and the order's response.
granules_metadata = client.query_cmr(params=my_params)

In [ ]:
orders = client.place_data_orders(params=my_params)
orders

#### Check Order Status
The following cell will show you the status of your data order. You can proceed in the notebook once all orders are "COMPLETE". If you proceed earlier only the completed data orders will be downloaded.

In [ ]:
for order in orders:
    status = client.order_status(order)
    print(order['dataset'], order['id'], status['status'])

### **Amy Note** :In my order email history, I get 5 emails, and they all say that 0 granules were processed. Yet I am able to download the data in the next step.

#### Download Data
Once all data orders are "COMPLETE", you can proceed downloading the data. The data are downloaded to the /data folder of this notebook directory.

In [ ]:
for order in orders:
    status = client.order_status(order)
    if status['status'] == 'COMPLETE':
        client.download_order(order)

#### 2.3. Downloading ICESat-2 data [directly] with *icepyx*
Behind the scenes, *IceFlow* is using the [*icepyx*](https://icepyx.readthedocs.io/en/latest/) Python package to download ICESat-2 data. *icepyx* is a standalone library that includes its own examples and documentation and welcomes contributions from data users (no previous GitHub or software development experience required!). Thus, it has a lot of additional functionality for querying, subsetting, ordering, and downloading ICESat-2 datasets (with in-the-works additions for data ingest into multiple formats), including making it easier to programmatically download data from multiple regions. Here we highlight some of the data visualization capabilities for exploring data prior to order and download.

In [ ]:
# Note from Jessica: I was unable to get Luis' code to work to create an icepyx query object, and for the visualizations
# we don't need the session info...

from iceflow.client import IceFlowClient
iceflow = IceFlowClient
iceflow.authenticate(user,password,email)
# the important bit
icepyx_session = iceflow.is2._session.session
# regular icepyx query
q = icepyx.Query(params)
# we need to assign the authenticated session in order to subset and download the granules.
q._session = icepyx_session
# q.download_granules()

In [ ]:
# Access the icepyx query object and import icepyx
import icepyx as ipx
bbox_list = [float(val) for val in (my_params["bbox"].split(","))]
is2_obj = ipx.Query(str(my_params["datasets"][-1]), bbox_list, [my_params["start"], my_params["end"]])

In [ ]:
# Visualize the query extent (this map won't be interactive if you don't have geoviews and the dev version of icepyx installed)
# Thus, for very small areas it can be difficult to see the specified region on a static world map (an area for future development!)
is2_obj.visualize_spatial_extent()

### 3. Working with the data
Now that we have downloaded our data, we need to make sure that they are in a common format to do analysis across missions.

Although typically we would include all import statements at the start of the workflow, here we have separated them into this section for instructional clarity.

### **Jessica update**: why are we ignoring warnings? I addressed the rest of the comments 

**Amy Note**: We should explain the libraries we're importing briefly. I generally think it's better practice to put all imports at the very top of the notebook but I don't have any documentation on hand to back that up! In particular we shuold explain why wer're ignoring warnings (and this could probably be in the same code block as the previous block). 

In [ ]:
import cartopy.crs as ccrs #geospatial (mapping) plotting library
import geopandas as gpd #add geospatial awareness/functionality to pandas
from iceflow.processing import IceFlowProcessing as ifp
%matplotlib widget
import matplotlib.pyplot as plt #Python visualization
import pandas as pd #data analysis and manipulation tool
import warnings #Python warnings module
warnings.filterwarnings("ignore")

In [ ]:
# Pre-IceBridge ATM granule data
preib_gdf = ifp.to_geopandas('data/atm1b_data_2020-11-15T20-05.hdf5')
# ICESat granule data
glas_gdf = ifp.to_geopandas('data/twaties-test-GLAH06-2000-2010.h5')
# ICESat-2 granule data
is2_gdf = ifp.to_geopandas('data/processed_ATL06_20181214041627_11690105_003_01.h5')

In [ ]:
# first, let's see what's in the harmonized dataframe and its shape.
display(preib_gdf.head(), preib_gdf.shape)

In [ ]:
# we do the same for the ICESat/GLAS dataframe
display(glas_gdf.head(), glas_gdf.shape)

In [ ]:
display(is2_gdf.head(), is2_gdf.shape)

In [ ]:
import geoviews as gv
gv.extension('bokeh')

# bbox_poly = gv.Path(line_geoms).opts(color="red", line_color="red")
tile = gv.tile_sources.EsriImagery.opts(width=500, height=500)

#                   vmin=0, vmax=1000, label=["preib","glad","is2"])
    
pts1 = gv.Points(preib_gdf, vdims=["elevation"]).opts(marker="plus")
pts2 = gv.Points(glas_gdf, vdims=["elevation"]).opts(marker="circle")
pts3 = gv.Points(is2_gdf, vdims=["elevation"]).opts(marker="diamond")

trackmap = tile * pts1 * pts2 * pts3
# trackmap.opts(colorbar=True)
trackmap

In [ ]:
map_fig=plt.figure()
map_ax=map_fig.add_subplot(111)

world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
world.plot(ax=map_ax, facecolor="lightgray", edgecolor="gray")
# MOA.show(ax=map_ax, cmap='gray', clim=[14000, 17000])

for onegdf, lab in zip([preib_gdf, glas_gdf, is2_gdf],["preib","glas","is2"]):
    ms=map_ax.scatter( onegdf["longitude"], onegdf["latitude"],  2, c=onegdf["elevation"], \
                  vmin=0, vmax=1000, label=lab)
# map_ax._aspect='equal'
plt.colorbar(ms, label='elevation');

---

Use code and example from https://nbviewer.jupyter.org/github/nicholas-kotlinski/2020_ICESat-2_Hackweek_Tutorials/blob/master/05.Geospatial_Analysis/shean_ICESat-2_hackweek_tutorial_GeospatialAnalysis_rendered.ipynb

## [Draft notebook for April 2021 Earthdata Webinar]

#### Learning Objectives (tbd)
* (goal from discussion) Introduce audience to the “Ice cubed” (ICESat/OIB/ICESat-2) missions and products
* (goal from discussion Promote icepyx and IceFlow as a way to address the need to access/harmonize multiple data products/resolutions/formats/structure


#### Webinar Outline
* Intro of harmonization challenge
    - Moving one level deeper into the need for time series analysis across disparate platforms
* Intro to the altimetry missions
    - Use the IceFlow intro notebook to guide this
    - Highlight the impressive time series
* Overview of IceFlow and icepyx
    - Audience and use cases of these tools
    - Icepyx genesis, open science and community development principles
        - working on that balance between community development and end use
    - IceFlow overview
    - How do these tools address challenges
        - IceFlow aims to make it easier to harmonize OIB/ICESat data with ICESat-2 using backend IceFlow API
* Use Case / Application
    - Leverage ICESat-2 hackweek topics
    - https://github.com/ICESAT-2HackWeek/geospatial-analysis/blob/master/shean_ICESat-2_hackweek_tutorial_GeospatialAnalysis.ipynb
* Short navigation of nsidc site, how to get to tools, notebook
    - Have Jennifer post links while we talk.
* Demos
    - **Use this notebook to pulls pieces of iceflow / icepyx using a specific science application**